### 네이버 연관검색어 데이터 수집
- 정적 페이지: html(str) -> BeauitifulSoup -> bs_obj.select(css-selector) -> text
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 1. 웹페이지 분석 : URL
query = "삼성전자"
url = f"https://search.naver.com/search.naver?query={query}"
print(url)

https://search.naver.com/search.naver?query=삼성전자


In [3]:
# 2. request(URL) > response : str(html)
response = requests.get(url)
response

<Response [200]>

In [4]:
response.text[:250]

'<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maxi'

In [5]:
# 3. str(html) > bs object
dom = BeautifulSoup(response.text, "html.parser")
type(dom)

bs4.BeautifulSoup

In [6]:
# 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)
# select() : list(Tag, Tag)
# select_one() : Tag
elements = dom.select(".lst_related_srch > .item")
len(elements)

10

In [7]:
element = elements[0]
keyword = element.select_one(".tit").text
keyword

'삼성전자 주가'

In [8]:
link = element.select_one("a").get("href")
link

'?where=nexearch&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90+%EC%A3%BC%EA%B0%80&ie=utf8&sm=tab_she&qdt=0'

In [9]:
# 각각의 엘리먼트에서 text 데이터 수집
keywords = []
for element in elements:
    keyword = element.text.strip()
    keywords.append(keyword)
print(keywords)

['삼성전자 주가', '삼성전자 배당금', '삼성전자 주식', '삼성전자 특별보너스', '삼성전자 배당금 지급일', '오늘 삼성전자 주가', '삼성전자 성과급', '삼성 전자레인지', '삼성전자 채용', '삼성전자 보너스']


In [10]:
keywords = [element.text.strip() for element in elements]
print(keywords)

['삼성전자 주가', '삼성전자 배당금', '삼성전자 주식', '삼성전자 특별보너스', '삼성전자 배당금 지급일', '오늘 삼성전자 주가', '삼성전자 성과급', '삼성 전자레인지', '삼성전자 채용', '삼성전자 보너스']


In [11]:
# 5. str(text) > DataFrame
df = pd.DataFrame({"keywors": keywords})
df["query"] = query
df.head()

,keywors,query
0,삼성전자 주가,삼성전자
1,삼성전자 배당금,삼성전자
2,삼성전자 주식,삼성전자
3,삼성전자 특별보너스,삼성전자
4,삼성전자 배당금 지급일,삼성전자


In [12]:
# 현재 시간 데이터 추가
from datetime import datetime

now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
now

'2022-02-25 16:03'

In [13]:
df["date_time"] = now
df.head()

,keywors,query,date_time
0,삼성전자 주가,삼성전자,2022-02-25 16:03
1,삼성전자 배당금,삼성전자,2022-02-25 16:03
2,삼성전자 주식,삼성전자,2022-02-25 16:03
3,삼성전자 특별보너스,삼성전자,2022-02-25 16:03
4,삼성전자 배당금 지급일,삼성전자,2022-02-25 16:03


In [19]:
# query를 입력하면 데이터 프레임을 출력하는 함수
def naver_relate_keyword(query):
    
    url = f"https://search.naver.com/search.naver?query={query}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(".lst_related_srch > .item")
    keywords = [element.text.strip() for element in elements]
    
    df = pd.DataFrame({"keywors": keywords})
    df["query"] = query
    
    # now = datetime.now()
    # now = now.strftime("%Y-%m-%d %H:%M")
    # df["date_time"] = now
    
    return df

query = "내장산"
df = naver_relate_keyword(query)
df

,keywors,query
0,내장산cc,내장산
1,내장산 케이블카,내장산
2,내장산 등산코스,내장산
3,내장산 날씨,내장산
4,내장산 복분자주,내장산
5,내장산 맛집,내장산
6,내장산국민여가캠핑장,내장산
7,내장산생태탐방원,내장산
8,내장산 최단코스,내장산
9,내장산관광호텔,내장산


In [15]:
dfs = []
queries = ["삼성전자", "LG전자"]

for query in queries:
    print(query, end=" ")
    df = naver_relate_keyword(query)
    dfs.append(df)
    
result = pd.concat(dfs, ignore_index=True)
result

삼성전자 LG전자 

,keywors,query
0,삼성전자 주가,삼성전자
1,삼성전자 배당금,삼성전자
2,삼성전자 주식,삼성전자
3,삼성전자 특별보너스,삼성전자
4,삼성전자 배당금 지급일,삼성전자
5,오늘 삼성전자 주가,삼성전자
6,삼성전자 성과급,삼성전자
7,삼성 전자레인지,삼성전자
8,삼성전자 채용,삼성전자
9,삼성전자 보너스,삼성전자


In [16]:
def relational_keyword(keyword):
    url = f"https://search.naver.com/search.naver?query={keyword}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(".lst_related_srch > li")
    return [element.select_one(".tit").text for element in elements]


In [17]:
keyword = "KT"
relational_keyword(keyword)

['kt 고객센터',
 'kt 인터넷',
 'kt 대리점',
 'kt 먹통',
 'kt 고객센터 전화번호',
 'kt 홈페이지',
 '케이티',
 'kt m모바일',
 'sk텔레콤',
 'kt 요금제']